In [1]:
import sys
import time
import json
from pathlib import Path
import subprocess

sys.path.append("..")

from core.capture import take_screenshot
from core.ocr import extract_text
from core.config import CAPTURE_INTERVAL

In [2]:
OUTPUT_FILE = Path("inferred_log.jsonl")

print("Saving inferred activity →", OUTPUT_FILE.resolve())

Saving inferred activity → C:\Users\GCV\dev\work\auto-timesheet-llm-poc\notebooks\inferred_log.jsonl


In [3]:
def infer_task_from_text(text):
    if not text or len(text.strip()) < 20:
        return ""

    short = " ".join(text.split()[:120])  # keep small context

    PROMPT = f"""
You are detecting what a user is doing on their computer based on OCR text.

OCR text may be noisy.

Return ONLY one short sentence describing the user's current activity.
No explanation. No extra text.

OCR:
{short}
"""

    result = subprocess.run(
        ["ollama", "run", "llama3.2:3b"],
        input=PROMPT.encode("utf-8"),
        capture_output=True
    )

    output = result.stdout.decode("utf-8", errors="ignore").strip()
    return output


In [4]:
print("Starting incremental activity tracking... Press CTRL+C to stop.\n")

last_text = ""
last_task = ""

try:
    while True:
        image_path, timestamp = take_screenshot()
        text = extract_text(image_path)

        # Skip if OCR text almost unchanged
        if text[:200] == last_text[:200]:
            print(f"[{timestamp}] No significant change → skipping")
            time.sleep(CAPTURE_INTERVAL)
            continue

        task = infer_task_from_text(text)

        if not task:
            print(f"[{timestamp}] No task inferred")
            time.sleep(CAPTURE_INTERVAL)
            continue

        record = {
            "timestamp": timestamp,
            "task": task
        }

        with open(OUTPUT_FILE, "a", encoding="utf-8") as f:
            f.write(json.dumps(record) + "\n")

        print(f"[{timestamp}] → {task}")

        last_text = text
        last_task = task

        time.sleep(CAPTURE_INTERVAL)

except KeyboardInterrupt:
    print("\nStopped.")


Starting incremental activity tracking... Press CTRL+C to stop.

[2026-02-17_12-01-27] → User is tracking activity in a Python Jupyter notebook.
[2026-02-17_12-02-45] → The user is reviewing Tesseract OCR documentation.
[2026-02-17_12-03-56] No significant change → skipping
[2026-02-17_12-04-58] → User is viewing a GitHub repository for an auto-timesheet system.
[2026-02-17_12-06-09] No significant change → skipping
[2026-02-17_12-07-10] → The user is setting up Tesseract OCR software.
[2026-02-17_12-08-20] → User is reviewing documentation for Tesseract OCR software.

Stopped.
